In [7]:
import numpy as np
# Importing Pandas Library 
import pandas as pd
import matplotlib.pyplot as plt
from preprocessing import *
import os
import joblib
from sklearn.preprocessing import MinMaxScaler
from utils import *

import tensorflow as tf
import keras
from tensorflow.keras.models import Model, load_model

from models.EEGNet import *

from sklearn.metrics import confusion_matrix

from preprocessing import *

import warnings
warnings.filterwarnings("ignore")

In [8]:
@keras.saving.register_keras_serializable(package="my_package", name="UpdatedIoU")
class UpdatedIoU(tf.keras.metrics.IoU):
  def __init__(self,
        num_classes,
        target_class_ids,
        name=None,
        dtype=None,
        ignore_class=None,
        sparse_y_true=True,
        sparse_y_pred=True,
        axis=-1
    ):
    super(UpdatedIoU, self).__init__(
        num_classes=num_classes,
        target_class_ids=target_class_ids,
        name=name,
        dtype=dtype,
        ignore_class=ignore_class,
        sparse_y_true=sparse_y_true,
        sparse_y_pred=sparse_y_pred,
        axis=axis
    )

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_pred = tf.math.argmax(y_pred, axis=-1)
    return super().update_state(y_true, y_pred, sample_weight)


In [9]:
model = load_model(r'./checkpoints/orthogonal_trainable_true.keras')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ eyebrows input      │ (None, 4, 128, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ left input          │ (None, 4, 128, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ right input         │ (None, 4, 128, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ both input          │ (None, 4, 128, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ teeth input         │ (None, 4, 128, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ eyebrows            │ (None, 64)        │      2,848 │ eyebrows          │
│ (Functional)        │                   │            │ input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ left (Functional)   │ (None, 64)        │      2,848 │ left input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ right (Functional)  │ (None, 64)        │      2,848 │ right input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ both (Functional)   │ (None, 64)        │      2,848 │ both input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ teeth (Functional)  │ (None, 64)        │      2,848 │ teeth input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 320)       │          0 │ eyebrows[0][0],   │
│ (Concatenate)       │                   │            │ left[0][0],       │
│                     │                   │            │ right[0][0],      │
│                     │                   │            │ both[0][0],       │
│                     │                   │            │ teeth[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1024)      │    328,704 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 512)       │    524,800 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 768)       │    393,984 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 128, 6)    │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ softmax             │ (None, 128, 6)    │          0 │ reshape[0][0]     │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,783,586 (14.43 MB)

 Trainable params: 1,260,928 (4.81 MB)

 Non-trainable params: 800 (3.12 KB)

 Optimizer params: 2,521,858 (9.62 MB)

In [10]:
label_name = ['eyebrows', 'left', 'right', 'both', 'teeth']

train_x = []
test_x = []
train_y = []
test_y = []
for label in label_name:
    dataset = np.load(f'./running/{label}.npz')

    print(dataset['train_x'].shape, dataset['train_y'].shape, dataset['test_x'].shape, dataset['test_y'].shape)
    train_x.append(dataset['train_x'][:51500])
    train_y.append(dataset['train_y'][:51500])
    test_x.append(dataset['test_x'][:12900])
    test_y.append(dataset['test_y'][:12900])

train_x = np.concatenate(train_x)
train_y = np.concatenate(train_y)
test_x = np.concatenate(test_x)
test_y = np.concatenate(test_y)


train_x = train_x.transpose((0, 2, 1))
test_x = test_x.transpose((0, 2, 1))

train_x = np.expand_dims(train_x, axis=-1)
test_x = np.expand_dims(test_x, axis=-1)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(65972, 128, 20) (65972, 128) (16493, 128, 20) (16493, 128)
(57701, 128, 20) (57701, 128) (14426, 128, 20) (14426, 128)
(53477, 128, 20) (53477, 128) (13370, 128, 20) (13370, 128)
(51868, 128, 20) (51868, 128) (12968, 128, 20) (12968, 128)
(51752, 128, 20) (51752, 128) (12939, 128, 20) (12939, 128)
(257500, 20, 128, 1) (257500, 128) (64500, 20, 128, 1) (64500, 128)


In [ ]:
y_train_pred = model.predict([
    train_x[:, :4], 
    train_x[:, 4:8], 
    train_x[:, 8:12],
    train_x[:, 12:16],
    train_x[:, 16:20]
])

y_test_pred = model.predict([
    test_x[:, :4], 
    test_x[:, 4:8], 
    test_x[:, 8:12],
    test_x[:, 12:16],
    test_x[:, 16:20]
])
y_train_pred.shape, y_test_pred.shape

In [ ]:
y_train_pred_argmax = np.argmax(y_train_pred, 2)
y_test_pred_argmax = np.argmax(y_test_pred, 2)

y_train_pred_argmax.shape, y_test_pred_argmax.shape

In [ ]:
iou_results = {}
for _class in range(5):
    iou_metric = tf.keras.metrics.IoU(num_classes=6, target_class_ids=[_class+1])
    print(f"Class [{label_name[_class]}] - Train:", iou_metric(train_y, y_train_pred_argmax))
    print(f"Class [{label_name[_class]}] - Test:", iou_metric(test_y, y_test_pred_argmax))
    iou_results[label_name[_class]] = [
        tf.get_static_value(iou_metric(train_y, y_train_pred_argmax)), 
        tf.get_static_value(iou_metric(test_y, y_test_pred_argmax))
    ]
    print(f"Class [{label_name[_class]}] - Train:", iou_results[label_name[_class]][0])
    print(f"Class [{label_name[_class]}] - Test:", iou_results[label_name[_class]][1])

In [ ]:
iou_results

In [ ]:
iou_results['Index'] = ['Train', 'Test']
pd.DataFrame.from_dict(iou_results).set_index('Index').T.plot(kind='bar')
plt.ylim(0.85, 0.96)

In [ ]:
cm_total = np.zeros((6, 6))

for y_t, y_p in zip(train_y, y_train_pred_argmax):
    cm = confusion_matrix(y_t, y_p, labels=[0, 1, 2, 3, 4, 5])
    cm = np.array(cm)
    cm_total = cm_total + cm

result = []
for cls in range(6):
    tp = cm_total[cls, cls]
    fn = np.sum(np.delete(cm_total[cls, :], cls))
    fp = np.sum(np.delete(cm_total[:, cls], cls))
    tn = np.delete(cm_total, cls, axis=0)
    tn = np.sum(np.delete(tn, cls, axis=1))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2 * precision * recall) / (precision + recall)
    acc = (tp + tn) / (tp + fn + tn + fp)
    specifity = tn/(tn+fp)

    result.append([precision, recall, f1, acc, specifity])

result = np.array(result)

print(f'precision, recall, f1, acc, specifity\n{result}')

In [ ]:
cm_total = np.zeros((6, 6))

for y_t, y_p in zip(test_y, y_test_pred_argmax):
    cm = confusion_matrix(y_t, y_p, labels=[0, 1, 2, 3, 4, 5])
    cm = np.array(cm)
    cm_total = cm_total + cm

result = []
for cls in range(6):
    tp = cm_total[cls, cls]
    fn = np.sum(np.delete(cm_total[cls, :], cls))
    fp = np.sum(np.delete(cm_total[:, cls], cls))
    tn = np.delete(cm_total, cls, axis=0)
    tn = np.sum(np.delete(tn, cls, axis=1))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2 * precision * recall) / (precision + recall)
    acc = (tp + tn) / (tp + fn + tn + fp)
    specifity = tn/(tn+fp)

    result.append([precision, recall, f1, acc, specifity])

result = np.array(result)

print(f'precision, recall, f1, acc, specifity\n{result}')

In [ ]:
auc = tf.keras.metrics.AUC()
auc_result = auc(train_y, y_train_pred_argmax)
auc_result

In [14]:
from sklearn.metrics import roc_curve

In [ ]:
train_y_onehot = tf.one_hot(train_y, 6)
train_y_onehot.shape


In [ ]:
fpr_keras, tpr_keras, thresholds_keras = roc_curve(train_y, y_train_pred_argmax)